# Model comparison

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the 1000 6x6 instances

In [2]:
# nodes_number = 36
# num_agents = [3, 6, 9, 12]

networks_df = pd.read_csv("data/d_it_ij_6x6_1000it.csv",
                          decimal=",",
                          header=None)

# print(networks_df.dtypes)
networks_df.head()

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,NaN,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,4.000000,...,27.000000,27.000000,28.000000,28.000000,28.000000,29.000000,29.000000,29.000000,30.000000,30.000000
1,NaN,7.000000,8.000000,7.000000,8.000000,9.000000,8.000000,9.000000,10.000000,9.000000,...,33.000000,34.000000,33.000000,34.000000,35.000000,34.000000,35.000000,36.000000,35.000000,36.000000
2,it1,1.612014,1.454776,0.376347,1.263488,1.675461,0.383242,1.325470,1.502733,0.959483,...,0.199034,1.911133,1.343548,0.238743,1.667782,1.273160,0.289610,1.523664,1.968861,1.483620
3,it2,0.345666,1.997958,1.022579,1.344409,0.335500,0.768256,0.724642,1.146472,0.726154,...,0.126858,0.794601,1.432874,0.123810,1.866517,1.411055,1.125656,1.531357,0.725753,0.617501
4,it3,1.059043,1.866680,1.672658,0.410748,1.204280,1.351971,1.502285,0.093011,0.149816,...,1.773551,0.360475,0.016188,1.866432,1.783597,1.205805,0.093973,1.818958,1.471756,1.272506


Arcs dataframe (without weights)

In [3]:
arcs_ij_df = networks_df.iloc[0:2, 1:].astype(int) - 1
arcs_ij_df.head()

,1,2,3,4,5,6,7,8,9,10,...,71,72,73,74,75,76,77,78,79,80
0,0,0,1,1,1,2,2,2,3,3,...,26,26,27,27,27,28,28,28,29,29
1,6,7,6,7,8,7,8,9,8,9,...,32,33,32,33,34,33,34,35,34,35


Prune network dataframe from useless datas

In [4]:
arcs_weights_df = networks_df.iloc[2:, 1:]
arcs_weights_df.head()

,1,2,3,4,5,6,7,8,9,10,...,71,72,73,74,75,76,77,78,79,80
2,1.612014,1.454776,0.376347,1.263488,1.675461,0.383242,1.325470,1.502733,0.959483,1.690435,...,0.199034,1.911133,1.343548,0.238743,1.667782,1.273160,0.289610,1.523664,1.968861,1.483620
3,0.345666,1.997958,1.022579,1.344409,0.335500,0.768256,0.724642,1.146472,0.726154,0.654525,...,0.126858,0.794601,1.432874,0.123810,1.866517,1.411055,1.125656,1.531357,0.725753,0.617501
4,1.059043,1.866680,1.672658,0.410748,1.204280,1.351971,1.502285,0.093011,0.149816,0.665625,...,1.773551,0.360475,0.016188,1.866432,1.783597,1.205805,0.093973,1.818958,1.471756,1.272506
5,0.297746,1.873162,1.144040,1.749892,0.975469,0.168817,0.553882,1.758605,1.824679,0.606143,...,1.398726,1.794652,0.178435,0.930916,0.497418,1.837166,1.315127,0.259269,1.973621,1.269510
6,1.296527,1.255480,1.777435,1.812928,1.990680,0.990032,0.467010,0.629513,0.962954,0.695574,...,0.499949,0.563903,0.976726,1.407947,0.320083,0.168943,0.432240,0.193984,1.377742,0.847393


Set up datas that will be used to compare models

In [6]:
Arc = namedtuple("Arc", ["i", "j", "d_ij"])


def sixXsix_1000_networks(arcs_ij_df, arcs_weights_df):
    for _, row in arcs_weights_df.iterrows():  # each network instance...
        yield [Arc(arcs_ij_df.iloc[0, col_idx],
                   arcs_ij_df.iloc[1, col_idx],
                   row.iloc[col_idx]) for col_idx in range(len(row))]  # ... has a set of arcs


nodes = np.unique(arcs_ij_df.values)
